In [563]:
import luigi
import pandas as pd
import os


## Création du dossier pour stocker les données nettoyées

In [564]:
# Vérifiez si le dossier n'existe pas déjà

dossier = 'CleanData'

if not os.path.exists(dossier):
    # Créez le dossier
    os.makedirs(dossier, mode=0o777)
    os.makedirs(os.path.join(dossier, "2017"), mode=0o777)
    os.makedirs(os.path.join(dossier, "2022"), mode=0o777)
    print("Dossier créé avec succès.")


Dossier créé avec succès.


## Extraire les données sur la délinquance

In [565]:
class ExtractCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/2017/communes.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        communes = pd.read_csv(self.file_path, sep=",", low_memory=False)
        partis = pd.read_csv(self.file_path1, sep=",")

        voix_columns = [col for col in communes.columns if col.startswith('Voix')]

        # Trouver l'index de la colonne ayant le plus de voix par ligne
        index_max_voix = communes[voix_columns].idxmax(axis=1)

        # Stock le nom du gagnant dans une colonne
        communes['Nom Gagnant'] = communes.apply(lambda row: row['Nom'] if pd.notnull(index_max_voix[row.name]) else None, axis=1)
        communes['Prénom Gagnant'] = communes.apply(lambda row: row['Prénom'] if pd.notnull(index_max_voix[row.name]) else None, axis=1)

        # Noms de colonnes a garder
        colonnes = ['Nom Gagnant', 'Prénom Gagnant', 'Code du département', 'Libellé du département', 'Code de la commune', 'Libellé de la commune']

        communes = communes[colonnes]

        # Merge du gagnant avec un partis politique
        communes = pd.merge(communes, partis, right_on=['Nom', 'Prenom'], left_on=['Nom Gagnant', 'Prénom Gagnant'], how='inner')

        communes.to_csv(self.output().path, index=False)


In [566]:
class ExtractRSA(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/2017/rsa.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        rsa = pd.read_csv(self.file_path, sep=",")

        #Filtre sur l'année 2017
        rsa = rsa.query('an == 2017')

        #Suppresion des valeurs null
        rsa = rsa.dropna()

        #renomes les colonnes
        rsa = rsa.rename(columns={"libgeo": "Libellé de la commune", 'alloc_rsa_com' : 'rsa'})
        
        median = rsa['rsa'].median()
        iqr = rsa['rsa'].quantile(0.75) - rsa['rsa'].quantile(0.25)
        lower_bound = rsa['rsa'].quantile(0.25) - (1.5 * iqr)
        upper_bound = rsa['rsa'].quantile(0.75) + (1.5 * iqr)

        rsa = rsa[(rsa['rsa'] >= lower_bound) & (rsa['rsa'] <= upper_bound)]
        
        rsa = rsa.drop(columns={"an"})
        
        rsa.to_csv(self.output().path, index=False)


In [567]:
class MergeCommuneRSA(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [ExtractCommunes(file_path="./RawData/communes.csv", file_path1="./RawData/partis.csv"),
                ExtractRSA(file_path="./RawData/rsa.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        rsa = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")

        merge_data = pd.merge(communes, rsa, on='Libellé de la commune', how='inner')

        merge_data.to_csv(self.output().path, index=False)

In [568]:
class ExtractRevenusPauvretes(luigi.Task):
    file_path = luigi.Parameter()

    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/revenus_pauvretes.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        revenus = pd.read_csv(self.file_path, sep=";")

        #renomes les colonnes
        revenus = revenus.rename(columns={"CODGEO": "codgeo", "NBMENFISC17" : "Nombre de ménages fiscaux", "NBPERSMENFISC17" : "Nombre de personnes dans les ménages fiscaux", "MED17" : "Niveau de vie"})
        
        #Suppresion des valeurs null
        revenus = revenus.dropna(subset=["Nombre de ménages fiscaux", "Nombre de personnes dans les ménages fiscaux", 'Niveau de vie'])

        colonnes = ["codgeo", "Nombre de ménages fiscaux", "Nombre de personnes dans les ménages fiscaux", 'Niveau de vie']
        revenus = revenus[colonnes]
        
        revenus.to_csv(self.output().path, index=False)


In [569]:
class MergeRevenusCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [MergeCommuneRSA(file_path="./CleanData/2017/communes.csv", file_path1="./CleanData/2017/rsa.csv"), ExtractRevenusPauvretes(file_path="./RawData/revenus_pauvretes.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        revenus = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")

        merge_data = pd.merge(communes, revenus, on='codgeo', how='inner')

        merge_data.to_csv(self.output().path, index=False)

In [570]:
class ExtractEmploi(luigi.Task):
    file_path = luigi.Parameter()

    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/emploi.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        emploi = pd.read_csv(self.file_path, sep=";")

        #renomes les colonnes
        emploi = emploi.rename(columns={"CODGEO": "codgeo", "P17_ACT1564" : "Nombre de personnes actives de 15 à 64 ans", "P17_CHOM1564" : "Nombre de chômeurs de 15 à 64 ans", "C17_ACT1564_CS5" : "Nombre d'employés actifs de 15 à 64 ans"})
        
        #Suppresion des valeurs null
        emploi = emploi.dropna(subset=["Nombre de personnes actives de 15 à 64 ans", "Nombre de chômeurs de 15 à 64 ans", "Nombre d'employés actifs de 15 à 64 ans"])

        colonnes = ["codgeo", "Nombre de personnes actives de 15 à 64 ans", "Nombre de chômeurs de 15 à 64 ans", "Nombre d'employés actifs de 15 à 64 ans"]
        emploi = emploi[colonnes]
        
        emploi.to_csv(self.output().path, index=False)


In [ ]:
class MergeEmloiCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [MergeRevenusCommunes(file_path="./CleanData/2017/revenus_pauvretes.csv", file_path1="./CleanData/2017/data.csv"), ExtractEmploi(file_path="./RawData/emploi.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        emploi = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")

        merge_data = pd.merge(communes, emploi, on='codgeo', how='inner')

        merge_data.to_csv(self.output().path, index=False)

## Exécuter toutes les tâches

In [571]:
class ReadAllcommunes(luigi.Task):
    def requires(self):
        return [MergeEmloiCommunes(file_path="./CleanData/2017/emploi.csv", file_path1="./CleanData/2017/data.csv")]
    
    def run(self):
        print("lancement")

    def output(self):
        return luigi.LocalTarget('result.txt')

In [572]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'False')

dossier = "./Cleancommunes/"
Restart = True

if Restart is False:
    # Parcourir tous les fichiers du dossier
    for fichier in os.listdir(dossier):
        chemin_fichier = os.path.join(dossier, fichier)
        # Supprimer le fichier
        os.remove(chemin_fichier)

luigi.build([ReadAllcommunes()], local_scheduler=False, no_lock=True)

DEBUG: Checking if ReadAllcommunes() is complete
DEBUG: Checking if MergeRevenusCommunes(file_path=./CleanData/2017/revenus_pauvretes.csv, file_path1=./CleanData/2017/data.csv) is complete
DEBUG: Checking if ExtractEmploi(file_path=./RawData/emploi.csv) is complete
INFO: Informed scheduler that task   ReadAllcommunes__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ExtractEmploi___RawData_emploi_805348f591   has status   PENDING
DEBUG: Checking if MergeCommuneRSA(file_path=./CleanData/2017/communes.csv, file_path1=./CleanData/2017/rsa.csv) is complete
DEBUG: Checking if ExtractRevenusPauvretes(file_path=./RawData/revenus_pauvretes.csv) is complete
INFO: Informed scheduler that task   MergeRevenusCommunes___CleanData_2017___CleanData_2017_d6b7462d1b   has status   PENDING
INFO: Informed scheduler that task   ExtractRevenusPauvretes___RawData_revenu_d6412a86d0   has status   PENDING
DEBUG: Checking if ExtractCommunes(file_path=./RawData/communes.csv, file_path1=./R

lancement


True